In [ ]:
import os
from flask import Flask, request, jsonify, render_template
from tensorflow import keras
from keras.preprocessing import image
import numpy as np

app = Flask(__name__)

# Load the saved model
model_path = r'D:\Durham\CNN Model\Model.h5'
model = keras.models.load_model(model_path)

# Create a dictionary to map class indices to vegetable names
class_map = {
    0: 'Bean',
    1: 'Bitter_Gourd',
    2: 'Bottle_Gourd',
    3: 'Brinjal',
    4: 'Broccoli',
    5: 'Cabbage',
    6: 'Capsicum',
    7: 'Carrot',
    8: 'Cauliflower',
    9: 'Cucumber',
    10: 'Papaya',
    11: 'Potato',
    12: 'Pumpkin',
    13: 'Radish',
    14: 'Tomato'
}


def predict_vegetable(image_path):
    img = image.load_img(image_path, target_size=(150, 150))
    img_array = image.img_to_array(img)
    img_array = np.expand_dims(img_array, axis=0)
    img_array /= 255.0

    prediction = model.predict(img_array)
    predicted_class = np.argmax(prediction[0])
    class_name = class_map[predicted_class]
    
    return class_name

# Define the root route to serve the index.html page
@app.route('/')
def home():
    return render_template('index.html')


@app.route('/predict', methods=['POST'])
def make_prediction():
    if 'file' not in request.files:
        return jsonify({'error': 'No file part'}), 400

    file = request.files['file']
    if file.filename == '':
        return jsonify({'error': 'No selected file'}), 400

    # Save the uploaded image temporarily
    image_path = 'temp_image.jpg'
    file.save(image_path)

    # Predict the vegetable using the CNN model
    predicted_class = predict_vegetable(image_path)

    # Remove the temporary image
    os.remove(image_path)

    return jsonify({'predicted_vegetable': predicted_class}), 200


if __name__ == '__main__':
    app.run(debug=False, use_reloader=False)
